<a href="https://colab.research.google.com/github/batuhanyndny/fer2013-cnn/blob/master/fer_2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt-get install git-lfs
!git clone "https://github.com/batuhanyndny/fer2013-cnn"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 6s (356 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-prec

In [6]:
cd fer2013-cnn/

/content/fer2013-cnn


In [7]:
!git lfs pull

Git LFS: (1 of 1 files) 287.13 MB / 287.13 MB


In [0]:
import pandas as pd
import numpy as np

In [12]:
dataset = pd.read_csv("fer2013.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


# **Preprocessing**

In [13]:
width, height = 48, 48 

data = dataset['pixels'].tolist()

X = []

for j in data:
    x = [int(i) for i in j.split(' ')]
    x = np.asarray(x).reshape(width,height)
    X.append(x.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

y = pd.get_dummies(dataset['emotion']).values

np.save('data',X)
np.save('labels', y)

print('completed...')


completed...


In [14]:
ls

cnn.py*   fer2013.csv*    labels.npy         README.md
data.npy  fer_2013.ipynb  preprocessing.py*  testing.py*


# **CNN**

In [31]:
from keras.models import Sequential 
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.losses import categorical_crossentropy
from keras.layers import DepthwiseConv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

label_count = 7
batch_size = 64
epochs = 300
width, height = 48 , 48 


x = np.load('data.npy')
y = np.load('labels.npy')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 19239)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.15, random_state = 1823)

np.save('X_test.npy', X_test)
np.save('y_test.npy', y_test)
#%%

model = Sequential()


model.add(DepthwiseConv2D(16, (2,2), depth_multiplier=3, activation='relu', input_shape=(width, height, 1), depthwise_regularizer=l2(0.007)))
model.add(DepthwiseConv2D(16, (3, 3), padding='same',activation='relu',depth_multiplier=3))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1), strides=(1,1)))

model.add(Dropout(0.5))

model.add(DepthwiseConv2D(16, (3, 3), padding='same', activation='relu',depth_multiplier=3))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(16, (3, 3), padding='same', activation='relu', depth_multiplier=3))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(16, (3, 3), padding='same', activation='relu', depth_multiplier=3))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Dropout(0.5))

model.add(DepthwiseConv2D(256, (3, 3), padding='same', activation='relu', depth_multiplier=3))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(256, (3, 3), padding='same', activation='relu', depth_multiplier=3))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(256, (3, 3), padding='same', activation='relu', depth_multiplier=3))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(label_count, activation='softmax'))

model.compile(loss=categorical_crossentropy, optimizer=Adam(lr=0.03),metrics=['accuracy'])

model.summary()

filepath = "modelcp.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
callbacks_list = [checkpoint, lr_reducer]

trained = model.fit(np.array(X_train), np.array(y_train), 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1, 
                    validation_data=(np.array(X_valid), np.array(y_valid)), 
                    shuffle=True,
                    callbacks=callbacks_list)

print("****************************************************************************")
print("train completed")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d_14 (Depthwi (None, 17, 17, 3)         771       
_________________________________________________________________
depthwise_conv2d_15 (Depthwi (None, 6, 6, 9)           2313      
_________________________________________________________________
batch_normalization_30 (Batc (None, 6, 6, 9)           36        
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 9)           0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 6, 6, 9)           0         
_________________________________________________________________
depthwise_conv2d_16 (Depthwi (None, 2, 2, 27)          6939      
_________________________________________________________________
batch_normalization_31 (Batc (None, 2, 2, 27)          108       
__________

ResourceExhaustedError: ignored

# **Testing**

In [0]:
from keras.preprocessing.image import img_to_array
import imutils
import cv2
from keras.models import load_model
import numpy as np
from keras.models import model_from_json

# parameters for loading data and images
detection_model_path = '/anaconda3/pkgs/libopencv-3.4.2-h7c891bd_1/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml' #gonna change
loaded_model = load_model("modelcp.h5")


# hyper-parameters for bounding boxes shape
# loading models
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_classifier = loaded_model
EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised",
 "neutral"]


#feelings_faces = []
#for index, emotion in enumerate(EMOTIONS):
   # feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1))

# starting video streaming
cv2.namedWindow('your_face')
camera = cv2.VideoCapture(0)
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE)
    
    canvas = np.zeros((250, 300, 3), dtype="uint8")
    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
        key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
                    # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (48,48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        
        preds = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = EMOTIONS[preds.argmax()]

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300)
                cv2.rectangle(canvas, (7, (i * 35) + 5),
                (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45,
                (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                              (0, 0, 255), 2)
#    for c in range(0, 3):
#        frame[200:320, 10:130, c] = emoji_face[:, :, c] * \
#        (emoji_face[:, :, 3] / 255.0) + frame[200:320,
#        10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)


    cv2.imshow('your_face', frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()
